In [1]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


In [2]:
# Criar objeto do ticker
ticker_data = yf.Ticker("MSFT")

In [2]:
# def get_data_history_yahoo(symbol : str, period : str, interval, start = '1900-01-01', end = datetime.now(), prepost : bool = True, debug : bool = True):
#     '''
#     Data collection from yahoo

#     __Parameters:__
#     period : str
#         Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
#     interval : str
#         Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
#     start: str
#         Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
#     end: str
#         Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
#     prepost : bool
#         Include Pre and Post market data in results? Default is False
#     debug: bool
#         If passed as False, will suppress error message printing to console.
#     '''
#     dt = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost, debug=debug)

#     return dt

# dt = get_data_history_yahoo(symbol="AAPL", period= "1d", interval = "1mo", start = "2024-10-01", end = datetime.now(), prepost = True, debug = True)


In [ ]:
#### Não Funciona ####
# # Acessar fast_info
# ticker_data.info


In [11]:
yahoo_symbol_recommendations = ticker_data.recommendations
yahoo_symbol_recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,13,37,5,0,0
1,-1m,13,37,5,0,0
2,-2m,13,37,5,0,0
3,-3m,14,39,3,0,0


In [10]:
# Dividendos
cashflow = ticker_data.cashflow


In [12]:
# Notícias
balance_sheet = ticker_data.balance_sheet
balance_sheet

,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Ordinary Shares Number,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Share Issued,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Net Debt,33315000000.0,12533000000.0,35850000000.0,43922000000.0
Total Debt,67127000000.0,59965000000.0,61270000000.0,67775000000.0
Tangible Book Value,121660000000.0,128971000000.0,87720000000.0,84477000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,75531000000.0,111256000000.0,104749000000.0,130256000000.0
Other Short Term Investments,57216000000.0,76552000000.0,90818000000.0,116032000000.0
Cash And Cash Equivalents,18315000000.0,34704000000.0,13931000000.0,14224000000.0
Cash Equivalents,6744000000.0,26226000000.0,5673000000.0,6952000000.0


In [13]:
yahoo_symbol_sustainability = ticker_data.sustainability
yahoo_symbol_sustainability

,esgScores
adult,False
alcoholic,False
animalTesting,False
catholic,None
coal,False
controversialWeapons,False
environmentPercentile,None
environmentScore,1.65
esgPerformance,LAG_PERF
furLeather,False


Acessar notícias relacionadas ao ativo (se disponíveis).

In [4]:
# Eventos corporativos
calendar = ticker_data.calendar
print(calendar)

{'Dividend Date': datetime.date(2025, 3, 13), 'Ex-Dividend Date': datetime.date(2025, 2, 20), 'Earnings Date': [datetime.date(2025, 1, 28), datetime.date(2025, 2, 3)], 'Earnings High': 3.47, 'Earnings Low': 3.04, 'Earnings Average': 3.14891, 'Revenue High': 70520000000, 'Revenue Low': 68488700000, 'Revenue Average': 68876283030}


In [12]:
# # Datas disponíveis para opções
# options_dates = ticker_data.options
# print("Datas de opções:", options_dates)

# # Obter opções para uma data específica
# options = ticker_data.option_chain(options_dates[0])
# print("Opções de compra (calls):", options.calls)
# print("Opções de venda (puts):", options.puts)

In [11]:
# # Demonstrativo financeiro
# financials = ticker_data.financials
# print("Demonstrativo financeiro:", financials)

# # Balanço patrimonial
# balance_sheet = ticker_data.balance_sheet
# print("Balanço patrimonial:", balance_sheet)

# # Fluxo de caixa
# cashflow = ticker_data.cashflow
# print("Fluxo de caixa:", cashflow)

Informações financeiras mais detalhadas da empresa.

In [13]:
# # Dados de sustentabilidade
# sustainability = ticker_data.sustainability
# print(sustainability)

None


Informações de ESG (Environmental, Social, Governance).

In [14]:
# # Ações em circulação
# shares = ticker_data.shares
# print("Ações em circulação:", shares)

- MSFT: No financials data found, symbol may be delisted
Ações em circulação: None


Obtenha informações sobre ações em circulação e histórico de splits.

In [16]:
# Principais investidores institucionais
inst_holders = ticker_data.institutional_holders
inst_holders


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Group Inc,673.64M,2024-09-30,0.0906,301298828821
1,Blackrock Inc.,561.48M,2024-09-30,0.0755,251134097171
2,State Street Corporation,289.5M,2024-09-30,0.0389,129485265633
3,"FMR, LLC",211.63M,2024-09-30,0.0285,94655860039
4,"Geode Capital Management, LLC",165.02M,2024-09-30,0.0222,73810645850
5,JP Morgan Chase & Company,137.44M,2024-09-30,0.0185,61471400753
6,Price (T.Rowe) Associates Inc,135.66M,2024-09-30,0.0182,60676696803
7,Morgan Stanley,124.04M,2024-09-30,0.0167,55481116921
8,Norges Bank Investment Management,97.07M,2024-06-30,0.0131,43417071233
9,Capital International Investors,76.78M,2024-09-30,0.0103,34342556236


Dados sobre investidores institucionais e sua participação.

In [19]:
# Principais investidores institucionais
holders = ticker_data.major_holders
holders


,Breakdown,Unnamed: 1
0,0.06%,% of Shares Held by All Insider
1,73.51%,% of Shares Held by Institutions
2,73.55%,% of Float Held by Institutions
3,6983,Number of Institutions Holding Shares
